<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

Se va a implementar un algoritmo para resolver la suma de matrices de dimension N*N. 

Se implementaran dos algoritmos, uno ejecuta y resuelve el calculo de manera secuencial sobre el mismo procesador y el segundo implementa la suma de los terminos de la matriz de forma paralela utilizando GPU. Para esto se implementan hilos en dos dimensiones, a partir de un calulo en el kernel podemos acceder a cada elemento de la matriz para poder realizar la operacion.


#1.1 Explicación del algoritmo

La suma de matrices[1] es una operación lineal que consiste en unificar los elementos de dos o más matrices que coincidan en posición dentro de sus respectivas matrices y que estas tengan el mismo orden.

En otras palabras, el sumatorio de una o más matrices es la unión de los elementos que tengan la misma posición dentro de las matrices y que estas tengan el mismo orden.

![](https://drive.google.com/uc?export=view&id=16M9RJRRsQELQqiC15qRqNC_bdQBfMc1_)

Para sumar dos matrices se debe verificar que el orden de estas sean iguales.

#2. Armado del ambiente

##2.1 Armado del ambiente en CPU

No se requiere la ejecucion previa de algun comando

##2.1 Armado del ambiente en GPU

1. Ir a Entornto de ejecucion -> Cambiar tipo de entorno de ejecucion y seleccionar la opcion GPU.
2. Se requiere la instalación del modulo de Cuda para Python.

In [ ]:
!pip install pycuda

#3. Desarrollo

##3.1 Desarrollo en CPU

In [22]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  10#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import sys

tiempoTotal = time.time() * 1000
try: 
  matriz_A = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A = matriz_A.astype(numpy.int32())
  matriz_B = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B = matriz_B.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño no puede ser negativo")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B)
print("---------------------------------------------------")

tiempoBucle = time.time() * 1000
matriz_R = numpy.zeros((tamMatriz, tamMatriz))  
matriz_R = matriz_R.astype(numpy.int32())

for i in range (0, tamMatriz):
  for j in range(0, tamMatriz):
    matriz_R[i][j] = matriz_A[i][j] + matriz_B[i][j]

print("Matriz R:\n",matriz_R)
print("---------------------------------------------------")

tiempoTotal = (time.time() * 1000) - tiempoTotal
tiempoBucle = (time.time() * 1000) - tiempoBucle
print("--------------------------------------------------")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

---------------------------------------------------
Matriz A:
 [[1234  250  498 4289 3438 1595 1738  386 1877  639]
 [3656 2748 2210   54 2575 2334 3615  815 4198 3503]
 [3777 3272  465  104  749 3364 2714  417  543  819]
 [2015 2100 1414 2784 1017   48 2281 3934 3837 4269]
 [4471 2161 3631 3470 1381 3899 3003 3201 1837 2585]
 [4920 1359 2782 3162 2115 1822 1414 3711 1613  530]
 [1788 4815 1373 4685 2532 4392 4507 1561  458 1296]
 [1613 2870 2980 4158  299 3825 2459 1068 4758 1121]
 [2642  893 3147  717 4266 4582 1353 1227 1918  358]
 [2760 1105 4032 3488  569 2386 1545 1880 4149 2420]]
---------------------------------------------------
Matriz B:
 [[2927  665 4550 2506 1749 4540 2707 1446 3662 4998]
 [2935 4248  121 4965 2003 3509 4631 3982 1613 2598]
 [ 894  696 1758 4161 3646   93 4475 1242 4451  633]
 [3568 1872 3920 1638 3204 3084 2250 3422 4283 4454]
 [ 789   76 1022 4438  394 1214 3533 3365 3251 1299]
 [1679 1781 3426  624 1571  993 1885 3561 1430 3852]
 [1610 3911 1069  169  91

##3.2 Desarrollo en GPU

In [23]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  10#@param {type: "number"}
# --------------------------------------------

import time

import numpy
import sys
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time()*1000)

try: 
  matriz_A_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A_cpu = matriz_A_cpu.astype(numpy.int32())
  matriz_B_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B_cpu = matriz_B_cpu.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño de la matriz no puede ser negativo")

try:  
  matriz_R_cpu = numpy.zeros((tamMatriz, tamMatriz)) 
  matriz_R_cpu = matriz_R_cpu.astype(numpy.int32()) 
  matriz_A_gpu = cuda.mem_alloc(matriz_A_cpu.nbytes)
  matriz_B_gpu = cuda.mem_alloc(matriz_B_cpu.nbytes)
  matriz_R_gpu = cuda.mem_alloc(matriz_R_cpu.nbytes)
  cuda.memcpy_htod(matriz_A_gpu, matriz_A_cpu)
  cuda.memcpy_htod(matriz_B_gpu, matriz_B_cpu)
  cuda.memcpy_htod(matriz_R_gpu, matriz_R_cpu)
except Exception:
  sys.exit("Error de asignacion de memoria para gpu")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A_cpu)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B_cpu)
print("---------------------------------------------------")

module = SourceModule("""
__global__ void matrixMultiplicationKernel(float* A, float* B, float* R,
                                           int N
                                           ) {

    int x = blockIdx.y*blockDim.y+threadIdx.y;
    int y = blockIdx.x*blockDim.x+threadIdx.x;

    if(y < N){
      R[x+ y*N] = A[x+ y*N]  +  B[x+ y*N];
    }
}
""")
kernel = module.get_function("matrixMultiplicationKernel")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (tamMatriz + dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (tamMatriz + dim_hilo_y-1) / dim_hilo_y )

print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

tiempo_kernel = time.time() * 1000

kernel(matriz_A_gpu, matriz_B_gpu, matriz_R_gpu,
       numpy.int32(tamMatriz),        
       block=( dim_hilo_x, dim_hilo_y, 1), 
       grid=(dim_bloque_x, dim_bloque_y,1)
       )

tiempo_kernel = (time.time() *1000) - tiempo_kernel

cuda.memcpy_dtoh(matriz_R_cpu, matriz_R_gpu)
tiempoTotal = (time.time() * 1000) - tiempoTotal
print("---------------------------------------------------")
print("Matriz resultado:\n",matriz_R_cpu)
print("---------------------------------------------------")
print("Tiempo TOTAL: ", tiempoTotal, "[ms]" )
print("Tiempo GPU  : ", tiempo_kernel, "[ms]" )

---------------------------------------------------
Matriz A:
 [[3565 2369  943  776 1778 2033  498  644  202 1480]
 [2768 1323 3031 1903   74 1196  297 3883 1249 2559]
 [3564  144 4847 2429  994  665  832 1380 1833   86]
 [ 144 2186 2149 1153 2321  393 4898 2295  179  674]
 [ 396 3655 3741 2874 4252 4328 2310 1802 4455 4578]
 [ 654 4504  218 1441  421 1068 3725 1321  467  332]
 [2110 4618 1118 4602  469 3049 3016 4593 2313 4209]
 [ 180 4340 2683  758 3473 1833 3906 4074 1956 2684]
 [4808 3670 4097 4023  166 2953 3512 2153  615  333]
 [3326 3281  345 4201 2403 1367 1821 4507  742  967]]
---------------------------------------------------
Matriz B:
 [[ 193 4781 3764  929 2630 2128  263 3311 1367 3716]
 [ 956 4786 3858 1004 2219  612 3856 3675 3357  588]
 [2742 2183 3215  769 3853  627 1306  871 2288 1677]
 [2597 1394 3730 2295  385 2584 1012 2008  138 3286]
 [1857 1176 3805  432 3021 3878 1155 4481 2228 4515]
 [1694 1686  444 1478 3476 3700 4949  246 2407 1379]
 [1582 3441 4729 2670 110

#4. Tabla de pasos

##4.1 Tabla de pasos CPU

Tabla de ejecucion de forma secuencial
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|tiempoBucle	       |Se toma el tiempo inicial del algoritmo presentado.
>CPU	|for…	for            |Se realiza el algoritmo.
>CPU	|tiempoBucle	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

##4.1 Tabla de pasos GPU

Tabla de ejecucion en multiples hilos de ejecucion
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|numpy.zeros         |Inicializacion de matriz de resultado.
>**GPU**  |cuda.mem_alloc        |Reserva de memoria de GPU.
>**GPU**  |cuda.memcpy_htod      |Copia de memoria de CPU a GPU.
>CPU	|SourceModule        |Definición de código del kernel.
>CPU	|module.get_function        |Generación de función del Kernel de GPU.
>CPU	|dim_hilo_x, dim_hilo_y	    | Calcula las dimensiones para la ejecuciòn de 2D.
>CPU	|tiempo_kernel	       |Toma el tiempo inicial del algoritmo presentado.
>**GPU**	|kernel()            |Ejecución del kernel
>CPU	|tiempo_kernel	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

#5. Conclusiones

El algoritmo no presenta mucha dificultad, lo interesante es que el primer algoritmo posee una complejidad O(N^2) y su rendimiento empeorara a medida que las dimensiones de la matriz sean mayores. Con la implementación paralela aumenta la eficiencia frente a la ejecución secuencial.

#6. Bibliografía

[1].Suma de matrices [matrices](https://economipedia.com/definiciones/suma-de-matrices.html)